<a href="https://colab.research.google.com/github/ShivprasadKomul02/Stock-Prediction-Using-Machine-Learning/blob/main/STOCK_MARKET_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mplcursors
!pip install yahoo_fin
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from tensorflow.python.keras.layers import Dense,LSTM
from tensorflow.python.keras import Sequential
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import plotly.express as px
import plotly.graph_objects as go
import mplcursors as mpl
from yahoo_fin import stock_info as si
import datetime


In [ ]:
#comapany = input('Company Name')

today = datetime.datetime.utcnow().date()
yesterday = today - datetime.timedelta(days=1)
df = web.DataReader('RELIANCE.NS', data_source='yahoo', start='2015-01-01', end=yesterday)

# fig = px.histogram(df, x=df['Close'])
# fig.show()
dates =[]
for x in range(len(df)):
    newdate = str(df.index[x])
    newdate = newdate[0:10]
    dates.append(newdate)



df['dates'] = dates

df['dates']

fig = go.Figure(data=[go.Candlestick(x=df['dates'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])
fig.show()

#fig = go.Figure(data=[go.Candlestick(open=df['Open'], high=df['High'],low=df['Low'], close=df['Close'])])


In [ ]:
si.get_quote_table('RELIANCE.NS',dict_result=False)
#si.get_live_price('INFY.NS')


In [ ]:
plt.figure(figsize=(18,9))
x = plt.axes()
x.set_facecolor('white')
plt.title('Close Price History')
plt.plot(df['Close'])
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price', fontsize=18)
plt.show()

In [ ]:
data = df.filter(['Close'])
#Convert the dataframe to a numpy array
dataset = data.values
#Get the number of rows to train the model on
training_data_len = math.ceil( len(dataset) * .8 )

training_data_len

1128

In [ ]:
#Scale the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

array([[0.02009979],
       [0.01948754],
       [0.01696041],
       ...,
       [0.99017802],
       [0.96420339],
       [0.94732106]])

In [ ]:
#Create the training data set
#Create the scaled training data set
train_data = scaled_data[0:training_data_len , :]
#Split the data into x_train and y_train data sets
x_train = []
y_train = []

for i in range(60, len(train_data)):
  x_train.append(train_data[i-60:i, 0])
  y_train.append(train_data[i, 0])
  if i<= 61:
    print(x_train)
    print(y_train)
    print()

In [ ]:

x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences= False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, batch_size=1, epochs=1)

In [ ]:
#Create the testing data set
#Create a new array containing scaled values from index 1543 to 2002 
test_data = scaled_data[training_data_len - 60: , :]
#Create the data sets x_test and y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
  x_test.append(test_data[i-60:i, 0])

In [ ]:
x_test = np.array(x_test)

In [ ]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

In [ ]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
rmse=np.sqrt(np.mean(((predictions- y_test)**2)))

In [ ]:
#Plot the data
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions
#Visualize the data
plt.figure(figsize=(19,9))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price', fontsize=18)
a=plt.plot(train['Close'])
b=plt.plot(valid[['Close', 'Predictions']])

mpl.cursor(a)
mpl.cursor(b)
plt.legend(['Train', 'Real', 'Predictions'], loc='lower right')
plt.show()

In [ ]:
valid